# 第９章 メモリベース型協調フィルタリング

<br>

> 利用者間型は，活動利用者と嗜好パターンが似ている利用者をまず見つけ，彼らが好むものを推薦する．一方，アイテム間型では，活動利用者が好むアイテムと類似したアイテムを推薦する．

というように別れるが，分かりやすいので一旦前者をベースに説明していく．

> 機械学習の観点からは k 近傍法 (k-nearest neighbor method) とみなせる

これについては後述の数式を見るとわかる．
1人のユーザーの嗜好(アイテム評価)を1本のベクトルで表現して，その類似度(ベクトル間の距離とか)を考慮して考えるから．


<br>

## 9.1 利用者間型メモリーベース法

利用者間型メモリベース協調フィルタリングの代表的な「手法」として，GroupLens てのがある．
で，これは，[MovieLens](https://movielens.org/home) っていう「サービス」に実装されている．


> レストランを探す場合のことを考えてみよう．
このとき，自分と食べ物の嗜好が似ている何人かの人に尋ねてみて，
彼らの意見をもとにどの店で食事をするか決めたりすることがあるだろう．

GroupLens のやってることは本当にこれに尽きる．より厳密には，
自分と嗜好が真逆なユーザーの意見も参考にする．このアイテム，Aさんが好きって言うなら俺は嫌いだわきっと．

<br>

### 文字や用語の設定

> 評価値行列 $R$ は利用者 $x \in X$ のアイテム $y \in Y$ への評価値 $r_{xy}$ を要素とする行列

> 5 段階のスコアを用いた採点法（5.2 節を参照）で獲得した嗜好データであれば，評価値の定義域は $\mathcal{R} = \{1,\ldots,5\}$

のとこについて．

$\mathcal{X} = \{1,\ldots,n\}$ は $n$人のユーザーの集合，$\mathcal{Y} = \{1,\ldots,m\}$ は $m$個のアイテムの集合として定義されているが，評価値 $r_{xy}$ は厳密には定義されていない気がする．

「明示的or暗黙的に取得した，ユーザ x がアイテム y を好む度合いを表していると思われる量的尺度の値」という感じか．
例えば...
- 明示的に獲得した評価値：MovieLens みたいに利用者がつけた rate の値(星の数)
- 暗黙的に獲得した評価値：クリック回数，そのページを眺めていた時間

他のノーテーションは本文参照．

<br>

### 類似度の計算(9.1式)

> 類似度とは，嗜好パターンがどれくらい似ているかを定量化

という定義なので，$R$ の行ベクトルの似ている度合いを測れれば良く，方法は色々ある．

パッと思いつくのはユークリッド距離だが，これは微妙．
なぜなら「大きさが違ってもベクトルの方向が同じなら似ている思考パターンと見なす」と考える方が自然だから．

なので，相関係数(9.1式)やコサイン類似度が使われる．

ただいずれにせよ「共通に評価しているアイテム」しか使えないという問題があり，たとえば，

> 共通に評価したアイテムが一つ以下ならば，Pearson 相関は計算できないので 0 とする．

は妥当性ない．共通アイテム数を考慮した改良もあるが一旦触れない．

<br>

### 嗜好の予測(9.2式)

(9.2)式の第2項は，
$$
\begin{aligned}
& \frac{\sum_{x \in \mathcal{X_y}} \rho_{ax}(r_{xy}-\bar{r}_x')}{\sum_{x \in \mathcal{X}_y}|\rho_{ax}|} \\
&= \sum_{x \in \mathcal{X}_y} \left\{
\frac{|\rho_{ax}|}{\sum_{x \in \mathcal{X}_y} |\rho_{ax}|}
\mathrm{sign}(\rho_{ax})(r_{xy}-\bar{r}_x')
\right\}
\end{aligned}
$$
って感じで式変形するとわかりやすい．具体的には...

$(r_{xy}-\bar{r}_x')$の部分は，$x$ さんが アイテム $y$ を
「他のアイテムと比べて(自分の付けた評価値の平均値と比べて)どのくらい好きか/嫌いか」の値．つまり，個人によって良い rate つけまくる人とか逆の人とかいるけど，調整を行ってそういうノイズを除去した上での$x$ さんのアイテム $y$ に対する評価値．

$\mathrm{sign}(\rho_{ax})$ の部分は，
アクティブユーザ $a$ さんと
嗜好(の方向)が同じ人の意見はそのまま受け取り，
嗜好(の方向)が逆の人の意見は逆にして受け取る，という役割．
嗜好が同じ人の「うーん平均より好きかな/嫌いかな」はそのまま参考にし，
逆の人の「うーん平均より好きかな/嫌いかな」は逆にして受け取る．

$\frac{|\rho_{ax}|}{\sum_{x \in \mathcal{X}_y} |\rho_{ax}|}$
の分子は「嗜好パターンに強い関連がある人(嗜好がめっちゃ同じあるいはめっちゃ逆な人)の意見は重視したいので大きい重みを課す」という意味．分母は「分子が極端な値とならないようにする，具体的には $(r_{xy}-\bar{r}_x')$ の最小値以上最大値以下の範囲に収まるようにする」ためだと思う．

上記3つをまとめると，(9.2)式の第2項は，サンプルユーザーが「自分と嗜好が逆なのか同じなのか」と「自分の嗜好パターンにどのくらい関連があるのか」を考慮した上で，サンプルユーザたちの「平均より好きかな/嫌いかな」をまとめたもの．
それを第1項の「自分がつけた評価値の平均値」に足し合わせて，それを「未評価のアイテム $y$ への評価値の予測値」としている．

より分かりやすく言うと...
1. 全ユーザに対して「アイテムy(固定)はあなたの中で(あなたの平均と比べて)どのくらい好き?」と聞く．
2. (ユーザと自分の類似度を考慮した上で)それを混ぜ合わせて「このアイテムは私の中で(私の平均と比べて)このくらい好きだろう」というのを求める．
3. それを「私の平均」に足すことで「このアイテム私はこのくらい好き」を推定する．

<br>

### 9.1.1 利用者間型メモリベース法の改良

> 利用者の近傍を使う改良もある．ここでの近傍とは，式 (9.1) の相関が大きな，すなわち活動利用者と類似した嗜好をもつ利用者の集合のことである．式 (9.2) の推定評価値は，アイテム y を評価済みの全ての利用者の評価に基づいているが，これを事前に計算した活動利用者の近傍のみに基づいて計算する．実験によれば，近傍利用者数がある程度以上になると，それ以降は増やしても予測精度は向上しない．よって，近傍利用者だけに計算を限定することで計算量を減らし，データベースの参照も抑制できるので，効率よく計算できるようになる．

これは k 近傍法に近い考え方で，確かに計算コストは下がるが，

> ただし，モデルベース法のモデルほど頻繁にする必要はないが，近傍は定期的に更新しなくてはならないので，純粋なメモリベース法の利点は部分的には失われる．また，新規の参加者については近傍を新たに計算する必要が生じ，脱退者が他の利用者の近傍利用者であれば予測精度の低下を招く．

という問題がある．それはそう．毎回のレコメンドの度に近傍計算しなおせば問題無いが計算がきついので，どのくらいメモリベースの利点を手放すのか考える必要あり．

また，**基本的に評価値行列 $R$ は欠損値だらけ**であり，それに対処するための改良もたくさんある．

欠損のない(共通して評価されている)アイテムのみを使うより，
補完してでも全てのアイテムを使った方が精度が上がる時もある．

- デフォルト投票(平均値補完)
- EMアルゴリズムによる補完

などが挙げられている．


<br>

## 9.2 アイテム間型メモリベース法

ユーザー $a$ のアイテム $y$ に対する嗜好を求めたいとすると...

- ユーザー間型：(評価値行列 $R$ の行ベクトルの類似度をもとに) ユーザー $a$ と嗜好が似ているユーザーを探し，その人がアイテム $y$ を好きかどうかを参考にする．
- アイテム間型：(評価値行列 $R$ の列ベクトルの類似度をもとに) アイテム $y$ とファン層が似ているアイテムを探し，それを ユーザー $a$ が好きかどうかを参考にする．

という感じなので，方法論は大体同じ．省略する．

別の角度から言うと，

- ユーザー間型：アクティブユーザーと嗜好が似ている他のユーザーが好きなアイテムをレコメンドする．
- アイテム間型：アクティブユーザーが好きなアイテムとファン層が似ているアイテムをレコメンドする.

となる．

ここで「アイテムのページを閲覧する＝そのアイテムが好きである」と仮定すると，アイテム間型の考え方を使って次の手順で一時的個人化のレコメンドをすることができる．

1. ユーザー $a$ がアイテム $z$ のページを開いたとする．
2. アイテム $z$ とファン層(評価値行列 $R$ の列ベクトル)が似ている他のアイテムを探す．
3. それをレコメンドする．
